In [1]:
from langchain.tools import tool

In [5]:
def add_2_nums(x: int, y: int) -> int:
    """
    Add two numbers and return the sum.

    This tool performs simple addition of two integers. It's useful for basic arithmetic operations 
    when you need to combine two numerical values. While simple, it ensures accurate addition without 
    the risk of floating-point errors that can occur with some manual calculations.

    Args:
        x (int): The first integer to be added. This can be any positive or negative whole number.
        y (int): The second integer to be added. This can be any positive or negative whole number.

    Returns:
        int: The sum of x and y. This will always be an integer.

    Note:
        - This function only works with integers. If you need to add floating-point numbers, you'll need to use a different tool.
        - The function does not have a limit on the size of integers it can handle beyond Python's built-in integer limits.
        - If the result of the addition exceeds the maximum integer size that can be represented in Python, an OverflowError may occur.
        - This tool does not perform any rounding or truncation; it returns the exact sum of the two input integers.
    """

    return x+y


In [6]:
tool(add_2_nums)

StructuredTool(name='add_2_nums', description="Add two numbers and return the sum.\n\nThis tool performs simple addition of two integers. It's useful for basic arithmetic operations \nwhen you need to combine two numerical values. While simple, it ensures accurate addition without \nthe risk of floating-point errors that can occur with some manual calculations.\n\nArgs:\n    x (int): The first integer to be added. This can be any positive or negative whole number.\n    y (int): The second integer to be added. This can be any positive or negative whole number.\n\nReturns:\n    int: The sum of x and y. This will always be an integer.\n\nNote:\n    - This function only works with integers. If you need to add floating-point numbers, you'll need to use a different tool.\n    - The function does not have a limit on the size of integers it can handle beyond Python's built-in integer limits.\n    - If the result of the addition exceeds the maximum integer size that can be represented in Python

In [1]:
from tools.python_interpreter import UVPythonInterpreter

In [2]:
interpreter = UVPythonInterpreter()

# Start a new chat session
result = interpreter.create_session()

INFO:tools.python_interpreter:UV version: uv 0.4.4
INFO:tools.python_interpreter:Created session directory: /home/bobby/Repos/llm_chatbot/llm_chatbot/llm_sandbox/0fed151a-37d0-44d7-aa4e-1ae45ee16a40
INFO:tools.python_interpreter:
INFO:tools.python_interpreter:Using Python 3.11.9 interpreter at: /usr/bin/python3.11
Creating virtualenv at: .venv
Activate with: source .venv/bin/activate

14.03s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
INFO:tools.python_interpreter:Session 0fed151a-37d0-44d7-aa4e-1ae45ee16a40 created successfully


In [3]:
result

{'status': 'success',
 'session_id': '5fdffdeb-c0a0-4872-a9e6-6ffce0ab9f73',
 'message': 'Session created successfully'}

In [4]:
session_id = result['session_id']
# Install a package in the session
interpreter.install_package(session_id, "numpy")

INFO:tools.python_interpreter:Running command in session 5fdffdeb-c0a0-4872-a9e6-6ffce0ab9f73: uv pip install numpy
INFO:tools.python_interpreter:Command output: 


{'status': 'success', 'output': ''}

In [5]:
# Run some code in the session
out = interpreter.run_python_code(session_id, """
import numpy as np
print(np.array([1, 2, 3]))
""")

print(out['output'])

INFO:tools.python_interpreter:Wrote code to /home/bobby/Repos/llm_chatbot/llm_chatbot/llm_sandbox/4dc4a791-3998-4bb3-8dac-531de344a3dd/temp_script.py
INFO:tools.python_interpreter:Running command in session 4dc4a791-3998-4bb3-8dac-531de344a3dd: /home/bobby/Repos/llm_chatbot/llm_chatbot/llm_sandbox/4dc4a791-3998-4bb3-8dac-531de344a3dd/.venv/bin/python /home/bobby/Repos/llm_chatbot/llm_chatbot/llm_sandbox/4dc4a791-3998-4bb3-8dac-531de344a3dd/temp_script.py
INFO:tools.python_interpreter:Command output: 
INFO:tools.python_interpreter:Removed temporary script: /home/bobby/Repos/llm_chatbot/llm_chatbot/llm_sandbox/4dc4a791-3998-4bb3-8dac-531de344a3dd/temp_script.py


In [ ]:
# List installed packages in the session
interpreter.list_installed_packages(session_id)

# When the chat is over, close the session
interpreter.close_session(session_id)

# If you need to run an isolated command outside of any session
output, error = interpreter.run_isolated_command("python --version")
print(output)

In [1]:
import nest_asyncio
nest_asyncio.apply()

In [9]:
import pexpect
session_dir = "/home/bobby/Repos/llm_chatbot/llm_chatbot/"
child = pexpect.spawn('/usr/bin/bash', cwd=session_dir)

586.37s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [23]:
import subprocess
import time
import select

class TerminalManager:
    def __init__(self):
        # Start the terminal process
        self.process = subprocess.Popen(
            ['bash'],  # Replace with ['sh'] for a POSIX shell, ['cmd'] for Windows command prompt
            stdin=subprocess.PIPE,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True,  # Use text mode for handling strings
            bufsize=1,  # Line-buffered
            universal_newlines=True  # Handle text mode (Python 3.7+)
        )

    def run_command(self, command, timeout=30):
        try:
            # Send a command to the terminal
            self.process.stdin.write(command + '\n')
            self.process.stdin.flush()

            output_lines = []
            error_lines = []

            # Set up a timeout for reading output
            start_time = time.time()
            while True:
                if time.time() - start_time > timeout:
                    self.process.kill()  # Terminate process if timeout is reached
                    raise TimeoutError("Command execution timed out")

                # Check if the process has terminated
                if self.process.poll() is not None:
                    break

                # Use select to handle non-blocking reads
                reads = [self.process.stdout, self.process.stderr]
                readable, _, _ = select.select(reads, [], [], 0.1)

                for fd in readable:
                    if fd == self.process.stdout:
                        output = self.process.stdout.readline()
                        if output:
                            output_lines.append(output.strip())
                    elif fd == self.process.stderr:
                        error = self.process.stderr.readline()
                        if error:
                            error_lines.append(error.strip())

            # Ensure all output is captured after process completion
            self.process.wait()
            return '\n'.join(output_lines), '\n'.join(error_lines)
        
        except Exception as e:
            # Handle exceptions and return error information
            return '', str(e)

    def close(self):
        # Close the terminal process
        self.process.stdin.close()
        self.process.terminate()
        self.process.wait()

import time  # For timeout functionality
import select  # For non-blocking I/O


In [24]:
manager = TerminalManager()

try:
    # Run some commands
    output, error = manager.run_command('apt update')
    print(f"Output: {output}")
    if error:
        print(f"Error: {error}")

finally:
    # Ensure the terminal is closed properly
    manager.close()

Output: 
Error: Command execution timed out
